In [11]:
from protera_stability.proteins import EmbeddingExtractor1D

In [17]:
from pathlib import Path
import pandas as pd

data_path = Path("../data")

!ls $data_path

raw			      stability_test_embeddings.pkl
stability.csv		      stability_test.h5
stability_diversity.csv       stability_train.csv
stability_embeddings.pkl      stability_train_diversity.csv
stability.h5		      stability_train_embeddings.pkl
stability_test.csv	      stability_train.h5
stability_test_diversity.csv


> * Los `.csv` deben contener las columnas `labels` y `sequences`.

> * `data_path` debe ser Path y un directorio que contenga tus `.csv`

In [19]:
pd.read_csv(data_path / "stability_train.csv")

,consensus_stability_score,sequence
0,0.37,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT
1,0.62,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK
2,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK
3,1.41,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT
4,1.11,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT
...,...,...
7705,0.80,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE
7706,0.82,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY
7707,0.66,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY
7708,1.05,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR


In [14]:
args_dict = {
    "model_name": "esm1b_t33_650M_UR50S",
    "base_path": data_path,
    "gpu": True,
}

emb_stabilty = EmbeddingExtractor1D(**args_dict)

Using cache found in /home/roberto/.cache/torch/hub/facebookresearch_esm_master


In [15]:
embeddings = emb_stabilty.generate_embeddings(
    ["stability_train.csv"],  # esta en data_path
    path_out="stability_embeddings_train",  # guardarlo en data_path/stability_train.pkl
    bs=256,
)

  0%|          | 0/31 [00:00<?, ?it/s]

[('GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT', 0.37), ('TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK', 0.62), ('TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK', -0.03), ('GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT', 1.41), ('TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT', 1.11), ('SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG', 1.24), ('TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY', 1.05), ('FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM', 0.89), ('TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT', 0.88), ('GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS', 1.15), ('SKDEAQREAERAIRSGNKEEARRILEEAGYSPEQAERIIEKLG', 1.15), ('GSSQETIEVEDEEEARRVAKELRKKGYEVKIERRGNKWHVHRI', 1.45), ('KLDEARELVERAKKEGTGVDVNGQRFEDWREAERWVREQEKNK', 0.78), ('GSSQETIEVEDEEEARRVAKILRKKGYEVKIERRGNKWHVHRT', 1.55), ('TTIHVGDLTLPYDNPKKAYEIAKKLAKKYNLTVTIKNGKITVT', 1.12), ('TWQDLVKIAEKALEKGEPITINGITVTTKEQAHQAIEYLKKAY', 1.04), ('GSSKTQYEYDMKEEAQKAYEKFKKQGIPVTITQKNGKWFVQVE', 1.54), ('GSSTTRYRFTDEEEARRAAKEWARRGYQVHVTQNGTWWEVEVR', 1.66), ('GTL

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
dset = emb_stabilty.generate_datasets(
    ["stability.csv"],
    h5_stem="stability",  # data_path / "stability".h5
    bs=256,
    embedding_file="stability_embeddings",  # cargar los embeddings desde donde se espera que esten
    target_name="stability_scores"
)

In [7]:
dset["labels"][:10], dset["sequences"][:10]

(array([ 0.37,  0.62, -0.03,  1.41,  1.11,  1.24,  1.05,  0.89,  0.88,
         1.15]),
 array([b'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT',
        b'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK',
        b'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK',
        b'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT',
        b'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG',
        b'TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY',
        b'FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT',
        b'GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS'], dtype=object))

In [8]:
dset.close() # hay que cerrarlo pq es un h5py